Stocks Rag

Author: Orvin Bellamy (https://github.com/orvinbellamy)

In [1]:
## Import libraries

import yfinance as yf
import pandas as pd
import openai
from openai import OpenAI
import os
import json
import fs
import time
from dotenv import load_dotenv
import logging as log
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from filehandler import FileHandler
from yfinancehandler import YFHandler
from eventhandler import EventHandler, ThreadManager
from agenthandler import AgentHandler
from stockanalyzer import analyze_stock, stock_data_setup
import importlib

In [2]:
## configruation

FILE_PATH = 'openai_upload_files/'
OPENAI_DIC_FILE_NAME = 'openai_files.json'

# Load schemas from JSON file
with open('config/dataframe_schemas.json', 'r') as f:
    schemas = json.load(f)

# Initialize an empty list to store the stocks
stocks_list = []

df_portfolio = pd.read_csv('config/portfolio.csv')

list_portfolio = list(df_portfolio['ticker'].unique())

# Open dic_files
with open(f'{FILE_PATH}{OPENAI_DIC_FILE_NAME}', 'r') as f:
    dic_files = json.load(f)

with open('config/assistants.json', 'r') as json_file:
    dic_assistants= json.load(json_file)

### Stocks Data

In [ ]:
analysis = analyze_stock(
	ticker= ['MSFT'], 
	dic_files=dic_files,
	dic_assistants=dic_assistants
)

In [ ]:
print(analysis)

In [3]:
ticker = ['MSFT']

client = OpenAI()

file_stocks = stock_data_setup(client=client, ticker=ticker, type='price', dic_files=dic_files)
file_cashflow = stock_data_setup(client=client, ticker=ticker, type='cash', dic_files=dic_files)
file_income_stmt = stock_data_setup(client=client, ticker=ticker, type='income', dic_files=dic_files)

# Have to manually update the tool_resources because the file_id can change
dic_assistants['fin_analyst']['tool_resources'] = {
	'code_interpreter': {'file_ids': [dic_files['df_stocks.csv']]}
}

with open('config/assistants.json', 'w') as json_file:
	json.dump(dic_assistants, json_file)
	print(f'assistants.json file has been updated')

fin_analyst = AgentHandler(
	client = client, 
	new=False,
	assistant_name = 'fin_analyst',
	dic_file = dic_assistants,
	dic_file_name = 'assistants.json',
	dic_file_path='config/'
	)

fin_consultant = AgentHandler(
	client = client,
	new=False,
	assistant_name = 'fin_consultant',
	dic_file = dic_assistants,
	dic_file_name = 'assistants.json',
	dic_file_path='config/'
	)

# prompt_start = f"This is your client. I want you to advice me on the stocks I list below.\
# 	I want to know the stocks' performance, if they're overvalued or undervalued, and whether or not they'll be a good investment.\
# 	I am looking for a long term (5+ years) investment and I have a relatively high risk tolerance.\
# 	Stocks: {ticker}"

d:\Documents\Python Repo\stock_rag\yfinancehandler.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)


file name: df_stocks.csv, file id: file-lQXETmraAeu3jMdlbkp2lFFh has been deleted
file name: df_stocks.csv is uploaded, new file id: file-1FfIHOC1xD0pHJUVx1wyHAp6
openai_upload_files/openai_files.json file has been updated


d:\Documents\Python Repo\stock_rag\yfinancehandler.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)


file name: df_cashflow.csv, file id: file-O50PA4SLwAt2xf00nPIqzAPn has been deleted
file name: df_cashflow.csv is uploaded, new file id: file-vsvH1v6wqI1FuiUYbMFevB8K
openai_upload_files/openai_files.json file has been updated


d:\Documents\Python Repo\stock_rag\yfinancehandler.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)


file name: df_income_stmt.csv, file id: file-OE7jjhuQeKm3IfXuzt39gYuB has been deleted
file name: df_income_stmt.csv is uploaded, new file id: file-40oMqdS45ZnEf47Mypr40FZ8
openai_upload_files/openai_files.json file has been updated
assistants.json file has been updated
Assistant has been updated, name: fin_analyst, id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
Assistant has been updated, name: fin_consultant, id: asst_mPlBQ4ZF8YIF238DWqWRhV4U


In [4]:
prompt_start = f"This is the financial consultant.\
	The client wants advice on the following stocks.\
	I need you to provide me your analysis on these stocks so I can provide the appropriate recommendations.\
	You will be provided with the data of these stocks, use them as you see fit.\
	Stocks: {ticker}"

thread = ThreadManager(
	client=client,
	prompt=prompt_start
)

thread.run_thread(
	assistant=fin_analyst,
	prompt=prompt_start,
	attachments=[file_stocks.file_id, file_cashflow.file_id, file_income_stmt.file_id]
)

next_prompt = thread.last_message

thread.run_thread(
	assistant=fin_consultant,
	prompt=next_prompt
)


assistant > Here are the information gathered from the financial analyst: 

These are the stocks the client wants advice on: 
1. MSFT

I will now review the files you have uploaded to gather relevant data (historical prices, income statements, cashflow reports) for Microsoft (MSFT). Let's start by examining the contents of the files to identify what type of data each contains.
assistant > code_interpreter

import pandas as pd

# File paths
file1_path = '/mnt/data/file-40oMqdS45ZnEf47Mypr40FZ8'
file2_path = '/mnt/data/file-vsvH1v6wqI1FuiUYbMFevB8K'
file3_path = '/mnt/data/file-1FfIHOC1xD0pHJUVx1wyHAp6'

# Load the files and inspect the first few rows of each to identify what data they contain
file1 = pd.read_csv(file1_path)
file2 = pd.read_csv(file2_path)
file3 = pd.read_csv(file3_path)

file1.head(), file2.head(), file3.head()
assistant > Here are the information gathered from the financial analyst: 

These are the stocks the client wants advice on: 
1. MSFT

The contents of the uploa

In [5]:
next_prompt = 'This message is for testing. Can you give a short sample response.'

thread.run_thread(
	assistant=fin_consultant,
	prompt=next_prompt
)


assistant > Based on the financial analysis, I recommend **BUY** for Microsoft (MSFT). Microsoft shows strong and stable financial performance, consistent cash flow, and significant investment in innovation. These elements make it a suitable addition to your investment portfolio for both stable returns and growth potential.get_last_message initiated
message_id: msg_7sfwJ8cSOB7BszPmVhzm0hFj, assistant_id: asst_mPlBQ4ZF8YIF238DWqWRhV4U
asst_id = blank
new_message: Based on the financial analysis, I recommend **BUY** for Microsoft (MSFT). Microsoft shows strong and stable financial performance, consistent cash flow, and significant investment in innovation. These elements make it a suitable addition to your investment portfolio for both stable returns and growth potential.
message_id: msg_X2lAydlkaiLYg2EXMmo1ecNj, assistant_id: None
message.assistant_id != asst_id, record previous message first
{'message_id': 'msg_7sfwJ8cSOB7BszPmVhzm0hFj', 'assistant_id': 'asst_mPlBQ4ZF8YIF238DWqWRhV4U'

In [10]:
thread.messages
client.beta.threads.messages.list(thread_id=thread.thread_id).data
# messages_data[0]
# type(file_cashflow)

[Message(id='msg_7sfwJ8cSOB7BszPmVhzm0hFj', assistant_id='asst_mPlBQ4ZF8YIF238DWqWRhV4U', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Based on the financial analysis, I recommend **BUY** for Microsoft (MSFT). Microsoft shows strong and stable financial performance, consistent cash flow, and significant investment in innovation. These elements make it a suitable addition to your investment portfolio for both stable returns and growth potential.'), type='text')], created_at=1725943605, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_vB5nUesXWxNQleChv6kdgiJu', status=None, thread_id='thread_v6TtUjmj6NhH2IQ0fsg8VrtG'),
 Message(id='msg_X2lAydlkaiLYg2EXMmo1ecNj', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='This message is for testing. Can you give a short sample response.'), type='text')], created_at=172594360

In [9]:
# existing_message_id = [dic['message_id'] for dic in thread.messages]

a = thread.messages[0]

thread.messages = []

thread.messages.append(a)

thread.get_last_message()
# thread.messages

get_last_message initiated
message_id: msg_7sfwJ8cSOB7BszPmVhzm0hFj, assistant_id: asst_mPlBQ4ZF8YIF238DWqWRhV4U
asst_id = blank
new_message: Based on the financial analysis, I recommend **BUY** for Microsoft (MSFT). Microsoft shows strong and stable financial performance, consistent cash flow, and significant investment in innovation. These elements make it a suitable addition to your investment portfolio for both stable returns and growth potential.
message_id: msg_X2lAydlkaiLYg2EXMmo1ecNj, assistant_id: None
message.assistant_id != asst_id, record previous message first
{'message_id': 'msg_7sfwJ8cSOB7BszPmVhzm0hFj', 'assistant_id': 'asst_mPlBQ4ZF8YIF238DWqWRhV4U', 'created_at': 1725943605, 'file_ids': [], 'role': 'assistant', 'run_id': 'run_vB5nUesXWxNQleChv6kdgiJu', 'message_text': 'Based on the financial analysis, I recommend **BUY** for Microsoft (MSFT). Microsoft shows strong and stable financial performance, consistent cash flow, and significant investment in innovation. These 

In [38]:
# existing_message_id = [dic['message_id'] for dic in thread.messages]
asst_id = ''

for message in messages_data:

	print(f'message_id: {message.id}, asst_id: {message.assistant_id}')

	if message.id in existing_message_id:
		print('id exists')

	else:
		# print('id doesnt exist')
		if asst_id == '':
			print('asst_id = blank')

			asst_id = message.assistant_id

		elif message.assistant_id == asst_id:

			print('message.assistant_id == asst_id, proceed normally')

		else:

			print('message.assistant_id != asst_id, start a new round of message')

message_id: msg_0YI1I3hF8OTzcEOG98L7Yi1V, asst_id: asst_mPlBQ4ZF8YIF238DWqWRhV4U
asst_id = blank
message_id: msg_QAPBtXXNgWnzX53d7SuW9dfQ, asst_id: None
message.assistant_id != asst_id, start a new round of message
message_id: msg_QVoKLLa1XshONulTZ3ZGpgwI, asst_id: asst_mPlBQ4ZF8YIF238DWqWRhV4U
message.assistant_id == asst_id, proceed normally
message_id: msg_H8VId2krDKV9oKD0G0ao59lF, asst_id: None
message.assistant_id != asst_id, start a new round of message
message_id: msg_Q5q5yM7L3n4Ng7F77GADFKON, asst_id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
message.assistant_id != asst_id, start a new round of message
message_id: msg_WJz18bE75m6GBxFfevGa0tgp, asst_id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
message.assistant_id != asst_id, start a new round of message
message_id: msg_itcdIzYvUErWcQaIbUeoNEAx, asst_id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
message.assistant_id != asst_id, start a new round of message
message_id: msg_uZbO2pgmoVS0p4i28XPKymwo, asst_id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
message.assistant_id != as

In [6]:
thread.delete_thread()

thread: thread_aLACFYn2LjezFmaC23UZlUAz has been deleted.
